# Fermi-Hubbard Hamiltonian
___
This section creates the Fermi-Hubbard Hamiltonian with OpenFermion

In [16]:
# from scipy import sparse
from scipy.sparse import linalg
from openfermion import fermi_hubbard, jordan_wigner, get_sparse_operator

In [17]:
"""Define the Hamiltonian."""
# Parameters from the paper

nsites = 4
U = 10.0

# Hopping Integral

# In the paper U/t = 4
t = U / 4

# Is the sign of the tunneling right?
fermionic_hamiltonian = fermi_hubbard(1, nsites, tunneling=-t, coulomb=U, periodic=False)

# Map to QubitOperator using the JW transform
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

# Convert to Scipy sparse matrix
hamiltonian_jw_sparse = get_sparse_operator(jw_hamiltonian)

# Compute ground energy
eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
ground_energy = eigs[0]

print(f"Ground State Energy: {ground_energy}")
print("JWT transformed Hamiltonian:")
print(jw_hamiltonian)


Ground State Energy: -2.4207019828364125
JWT transformed Hamiltonian:
(10+0j) [] +
(-0.5+0j) [X0 Z1 X2] +
(-0.5+0j) [Y0 Z1 Y2] +
(-2.5+0j) [Z0] +
(2.5+0j) [Z0 Z1] +
(-0.5+0j) [X1 Z2 X3] +
(-0.5+0j) [Y1 Z2 Y3] +
(-2.5+0j) [Z1] +
(-0.5+0j) [X2 Z3 X4] +
(-0.5+0j) [Y2 Z3 Y4] +
(-2.5+0j) [Z2] +
(2.5+0j) [Z2 Z3] +
(-0.5+0j) [X3 Z4 X5] +
(-0.5+0j) [Y3 Z4 Y5] +
(-2.5+0j) [Z3] +
(-0.5+0j) [X4 Z5 X6] +
(-0.5+0j) [Y4 Z5 Y6] +
(-2.5+0j) [Z4] +
(2.5+0j) [Z4 Z5] +
(-0.5+0j) [X5 Z6 X7] +
(-0.5+0j) [Y5 Z6 Y7] +
(-2.5+0j) [Z5] +
(-2.5+0j) [Z6] +
(2.5+0j) [Z6 Z7] +
(-2.5+0j) [Z7]


# Qulacs Simulation

In [21]:
"""Convert OpenFermion Hamiltonian to Qulacs Observable"""

from qulacs.observable import create_observable_from_openfermion_text
from openfermion.utils import count_qubits

qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

n_qubits = count_qubits(jw_hamiltonian)


print(f"Succesfully created Qulacs Hamiltonian from OpenFermion Hamiltonian on {n_qubits} qubits")


Succesfully created Qulacs Hamiltonian from OpenFermion Hamiltonian on 8 qubits


In [22]:
from qulacs import QuantumState

state = QuantumState(n_qubits)
state.set_Haar_random_state()

approx_ground_state = qulacs_hamiltonian.solve_ground_state_eigenvalue_by_power_method(state, 1000)
print(approx_ground_state)

(-2.392084789084998+0j)
